# Append all data files exported from BigQuery to CSVs in GCS

### Assumptions
* label is the first column in the exported files
* all other columns can be labeled x_{n} where number is the position of the column in relation to the label column

In [ ]:
#####################################################################
#
# extract table from from BQ to GCS CSVs
#
#####################################################################

In [4]:
# set params
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
REGION = "us-central1"

In [5]:
BQ_DATASET = "ds_central1"
BQ_TABLE = "tab_class_732inps_1600krows_tra"
TRAINING_DATA_BUCKET = "ap-alto-ml-1000-bucket-us-central1"

In [57]:
! bq extract \
  --destination_format=CSV \
  --field_delimiter=',' \
  --print_header=false \
  {BQ_DATASET}.{BQ_TABLE} \
  gs://{TRAINING_DATA_BUCKET}/{BQ_TABLE}_*.csv

Waiting on bqjob_rbddeba452ee2275_00000185a8933615_1 ... (108s) Current status: DONE   


In [20]:
def get_col_count(PROJECT_ID, REGION, BQ_DATASET, BQ_TABLE):
    from google.cloud import bigquery
    client = bigquery.Client(location=REGION, project=PROJECT_ID)    
    query = f"SELECT count(*) FROM `{PROJECT_ID}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMNS` WHERE table_name = '{BQ_TABLE}'"
    print(query)
    query_job = client.query(query, location=REGION, )
    return query_job.to_dataframe().values.tolist()[0][0]

bq_table_column_count = get_col_count(PROJECT_ID, REGION, BQ_DATASET, BQ_TABLE)
bq_table_column_count

SELECT count(*) FROM `ap-alto-ml-1000.ds_central1.INFORMATION_SCHEMA.COLUMNS` WHERE table_name = 'tab_class_732inps_1600krows_tra'


733

In [ ]:
#####################################################################
#
# combine CSVs into a single file
#
#####################################################################

In [15]:
import csv
from google.cloud import storage

In [ ]:
# write file containing column headers
l = list()
l.append("label")
for i in range(bq_table_column_count):
    l.append(f"x_{i}")


file_with_headers = f"{BQ_TABLE}_headers.csv"
with open(file_with_headers, 'w', newline='') as csvfile:
    w = csv.writer(csvfile, delimiter=',')
    w.writerow(l)
    
# copy file to GCS dir with data
! gsutil cp {file_with_headers} gs://{TRAINING_DATA_BUCKET}/{file_with_headers}

In [61]:
# get source files for GCS compose
storage_client = storage.Client()
bucket = storage_client.bucket(TRAINING_DATA_BUCKET)

sources = list()
sources.append(bucket.get_blob(file_with_headers))

# get blob names from GCS
blobs = bucket.list_blobs()
for b in bucket.list_blobs():
    if b.name != file_with_headers:
        sources.append(bucket.get_blob(b.name))
    else:
        pass

# chunk for GCS compose - max num files allowed at one time is 32
n = 30
source_lists = [sources[i:i + n] for i in range(0, len(sources), n)]

if len(source_lists) == 1:
    destination_blob_name = f"{BQ_TABLE}.csv"
    destination = bucket.blob(destination_blob_name)
    destination.content_type = "text/plain"
    destination.compose(source_lists[0])
else:
    for idx, source_list in enumerate(source_lists):
        if idx==0 and idx < len(source_lists)-1:
            destination_blob_name = f"{BQ_TABLE}_temp_{idx}.csv"
            destination = bucket.blob(destination_blob_name)
            destination.content_type = "text/plain"
            destination.compose(source_list)
        if idx > 0 and idx < len(source_lists)-1:
            last_temp = [bucket.get_blob(destination.name)]
            destination_blob_name = f"{BQ_TABLE}_temp_{idx}.csv"
            destination = bucket.blob(destination_blob_name)
            destination.content_type = "text/plain"
            destination.compose( last_temp + source_list)
        elif idx == len(source_lists)-1:
            last_temp = [bucket.get_blob(destination.name)]
            destination_blob_name = f"{BQ_TABLE}.csv"
            destination = bucket.blob(destination_blob_name)
            destination.content_type = "text/plain"
            destination.compose( last_temp + source_list)

In [66]:
GCS_TRAIN_URI = f"gs://{TRAINING_DATA_BUCKET}/{BQ_TABLE}.csv"